In [23]:
import sys
sys.path.append("../")

In [24]:
import pandas as pd
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [25]:
df_raw = pd.read_csv("../data/candles/AUD_CAD_H1.csv")

In [26]:
df_raw.shape

(67812, 15)

In [27]:
df_an = df_raw.copy()
df_an.reset_index(drop=True, inplace=True)

In [28]:
df_an.shape

(67812, 15)

In [29]:
df_an = RSI(df_an)

In [30]:
df_an = apply_patterns(df_an)

In [31]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [32]:
df_an.columns

Index(['Unnamed: 0', 'time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
       'bid_o', 'bid_h', 'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c',
       'RSI_14', 'body_lower', 'body_upper', 'body_bottom_percentage',
       'body_top_percentage', 'body_percentage', 'direction', 'body_size',
       'low_change', 'high_change', 'body_size_change', 'mid_point',
       'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_percentage_prev', 'body_percentage_prev_2',
       'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING',
       'TWEEZER_TOP', 'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'DOJI',
       'HAMMER', 'INVERTED_HAMMER', 'DARK_CLOUD_COVER', 'PIERCING_PATTERN',
       'BULLISH_ENGULFING', 'BULLISH_HARAMI', 'BEARISH_ENGULFING',
       'BEARISH_HARAMI', 'THREE_WHITE_SOLDIERS', 'THREE_BLACK_CROWS',
       'BULLISH_ABANDONED_BABY', 'BEARISH_ABANDONED_BABY', 'BULLISH_TRI_STAR',
       'EMA_200'],
      dtype='object'

In [33]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l',
            'mid_c', 'ask_c', 'bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [34]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim = df_slim.reset_index(drop=True)

In [35]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2013-01-17 04:00:00+00:00,1.03766,1.03776,1.03646,1.03726,1.03737,1.03715,False,-1,1.039146,29.674797
1,2013-01-17 05:00:00+00:00,1.03724,1.03740,1.03616,1.03684,1.03697,1.03670,False,-1,1.039119,28.112965
2,2013-01-17 06:00:00+00:00,1.03680,1.03774,1.03675,1.03748,1.03763,1.03734,True,1,1.039100,32.203390
3,2013-01-17 07:00:00+00:00,1.03750,1.03752,1.03648,1.03701,1.03713,1.03689,False,-1,1.039076,25.061728
4,2013-01-17 08:00:00+00:00,1.03700,1.03870,1.03694,1.03842,1.03855,1.03828,True,1,1.039069,34.871245


In [36]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0


def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE

In [37]:

df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [38]:
df_slim['SIGNAL'].value_counts()

SIGNAL
 0    62260
-1     2734
 1     2619
Name: count, dtype: int64

In [39]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5


def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * PROFIT_FACTOR + row.mid_c
    else:
        return 0.0


def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [40]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [41]:
df_slim[df_slim.SIGNAL == SELL].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
228,2013-01-30 11:00:00+00:00,1.04674,1.04686,1.04534,1.04536,1.04546,1.04525,True,-1,1.047518,29.388298,-1,1.043290,1.04674
232,2013-01-30 15:00:00+00:00,1.04628,1.04645,1.04487,1.04570,1.04584,1.04557,True,-1,1.047449,32.005141,-1,1.044830,1.04628
255,2013-01-31 14:00:00+00:00,1.04388,1.04410,1.04204,1.04243,1.04252,1.04234,True,-1,1.046660,35.494155,-1,1.040255,1.04388
258,2013-01-31 17:00:00+00:00,1.04260,1.04260,1.04043,1.04077,1.04089,1.04065,True,-1,1.046512,43.121150,-1,1.038025,1.04260
261,2013-01-31 20:00:00+00:00,1.04086,1.04134,1.03998,1.04010,1.04024,1.03996,True,-1,1.046332,36.081370,-1,1.038960,1.04086


In [42]:
df_plot = df_slim.iloc[0:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x=trades.sTime,
        y=trades[markers[i]],
        mode='markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=550)

In [43]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0

    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price

    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)

In [44]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]

    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))

In [45]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [46]:
df_results.result.sum()

332.0